## Loading the Data

In [1]:
from datasets import load_from_disk
dataset = load_from_disk('ape-ds')

In [2]:
dataset['train'][0]

{'bo': 'ཞི་ཁྲོ་སྤྲུལ་པའི་སྐུ་ལ་ཕྱག་འཚལ་ལོ༔',
 'en': 'Nirmāṇakāya peaceful and wrathful: to you I pay homage!',
 'topic': 'Confession, Termas, Tibetan Masters, Nyala Pema Dündul',
 'for-post-edit': 'Homage to the tathāgatas and the Victorious Ones of the Victorious Ones: homage to you!'}

## Load Unfinetuned Tokenizer, Model, and Data Collator

In [3]:
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM

checkpoint = "google-t5/t5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, device_map="cuda:0")
tokenizer = AutoTokenizer.from_pretrained('my-tokenizer')
model.resize_token_embeddings(len(tokenizer))
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

## Preprocess Data

The dataset can now be tokenized for training.

In [4]:
def ape_preprocess_function(examples):

    # Prepare translation inputs and targets
    inputs = ['Post-Edit Translation: ' + example for example in examples['for-post-edit']]
    targets = [example for example in examples['en']]
    
    # Tokenize translation inputs and targets
    model_inputs = tokenizer(inputs, text_target=targets, 
                                         max_length=256, truncation=False, padding="max_length")
    
    return model_inputs

def trans_preprocess_function(examples):

    # Prepare translation inputs and targets
    inputs = ['Translate Tibetan to English: ' + example for example in examples['bo']]
    targets = [example for example in examples['en']]
    
    # Tokenize translation inputs and targets
    model_inputs = tokenizer(inputs, text_target=targets, 
                                         max_length=256, truncation=False, padding="max_length")
    
    return model_inputs

In [5]:
from datasets import concatenate_datasets

tokenized_ape_dataset = dataset.map(ape_preprocess_function, batched=True)
tokenized_trans_dataset = dataset.map(trans_preprocess_function, batched=True)


Map:   0%|          | 0/21506 [00:00<?, ? examples/s]

In [6]:
from datasets import DatasetDict

tokenized_dataset = DatasetDict()

tokenized_dataset['train'] = concatenate_datasets([tokenized_ape_dataset['train'], tokenized_trans_dataset['train']])
#tokenized_dataset['test'] = concatenate_datasets([tokenized_ape_dataset['test'], tokenized_trans_dataset['test']])

## Train the Model

Finally, we can train the model. Note that the optimizer used is Adafactor. This is the optimizer that is preferred for translation tasks and for the T5 model in general. The transformers api includes a built in version of Adafactor, but I define it separately here so that we can optimize it with the 'accelerate' library.

In [7]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, Adafactor, EarlyStoppingCallback
from accelerate import Accelerator

accelerator = Accelerator()

optimizer = Adafactor(
    model.parameters(), 
    scale_parameter=True, 
    relative_step=False, 
    warmup_init=False, 
    lr=3e-4
)

model, optimizer = accelerator.prepare(model, optimizer)

In [8]:
import numpy as np
import evaluate

# Load BLEU and CHRF metrics
bleu_metric = evaluate.load("sacrebleu")
chrf_metric = evaluate.load("chrf")
ter_metric = evaluate.load("ter")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    
    # Decode predictions and labels
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Postprocess text
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # Compute BLEU score
    bleu_result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    bleu_score = bleu_result["score"]

    # Compute CHRF score
    chrf_result = chrf_metric.compute(predictions=decoded_preds, references=decoded_labels)
    chrf_score = chrf_result["score"]

    # Compute TER score
    ter_result = ter_metric.compute(predictions=decoded_preds, references=decoded_labels)
    ter_score = ter_result["score"]

    # Return rounded results
    metrics = {
        "bleu": round(bleu_score, 4),
        "chrf": round(chrf_score, 4),
        "ter": round(ter_score, 4)
    }

    #print("Computed Metrics:", metrics)

    return metrics

In [9]:
%env WANDB_PROJECT=ape-experiment

env: WANDB_PROJECT=ape-experiment


In [11]:
training_args = Seq2SeqTrainingArguments(
    output_dir=f"dual-model",
    auto_find_batch_size=True,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=False,
    eval_strategy='epoch',
    save_strategy='epoch',
    num_train_epochs=10
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset={'translation': tokenized_trans_dataset['test'], 'post-edit': tokenized_ape_dataset['test']},
    tokenizer=tokenizer,
    optimizers=(optimizer, None),
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

/tmp/ipykernel_4617/2482796188.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss,Translation Loss,Translation Bleu,Translation Chrf,Translation Ter,Post-edit Loss,Post-edit Bleu,Post-edit Chrf,Post-edit Ter
1,0.167500,No log,0.170563,10.264300,23.951300,100.361300,0.162716,21.460300,35.151800,83.319900
2,0.133600,No log,0.126804,20.757400,35.261900,82.559000,0.138191,22.502400,35.349600,80.900000
3,0.111500,No log,0.102051,28.934900,43.458500,72.932800,0.121697,23.187900,36.344700,80.877800
4,0.091200,No log,0.086048,35.142300,48.950100,66.033300,0.109127,25.329700,37.592800,78.237800
5,0.081200,No log,0.075692,40.594200,52.876400,60.245800,0.100380,26.560500,38.919400,77.851300
6,0.072200,No log,0.068580,44.521300,56.608100,56.058900,0.093610,28.098800,39.895900,76.041900
7,0.067900,No log,0.063813,48.022000,59.376000,52.604600,0.088778,29.197100,40.931300,75.276000
8,0.064100,No log,0.060741,50.389200,61.374900,50.308800,0.085367,30.541200,41.802300,73.769400
9,0.059500,No log,0.058941,52.066100,62.720200,48.660900,0.083466,31.172600,42.481200,73.338400
10,0.059700,No log,0.058384,52.910800,63.009800,47.872700,0.082719,31.611300,42.746100,72.735000


TrainOutput(global_step=483860, training_loss=0.09868416666950067, metrics={'train_runtime': 95598.2227, 'train_samples_per_second': 40.491, 'train_steps_per_second': 5.061, 'total_flos': 2.6194593624096768e+17, 'train_loss': 0.09868416666950067, 'epoch': 10.0})